## CE9010 Introduction to Data Science Project

# Data Scraper



First, we import the relevant libraries.

In [2]:
from bs4 import Comment, BeautifulSoup as bs
import urllib.request
import csv

3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]


# TO BE FORMATTED \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\


Data Acquisition
 ------

#### Warning, long runtime of up to 40 minutes. Program will finish scraping when END OF YEAR: 2016 is printed.

Scraper written in beautifulsoup.

#### Types of data extracted
---

Our main goal of our is to predict the impact of college prospects in the first and second year of NBA based on their college stats. Hence we choose to use their advanced stats available in college (ws, ws/40min) to generalize the impact on NBA measured in their advanced stats available in the NBA (per, ws, ws/48, bpm, vorp) for their first two years in the league. We chose to extract data from 1996 as college advanced statistics were only available from the 1995 season onwards. We also chose to only include the first 30 draft picks of each draft as players beyond the 30th picks usually do not play significant minutes in their teams.

Player's NBA stats extracted from [Basketball Reference](https://www.basketball-reference.com/).

Player's College stats extracted from [Sports Reference](https://www.sports-reference.com/)

---



#### Problems encountered

---

The main problem encountered when running the scraper is the long runtime for a relatively small dataset est <200? confirm again>. The bottleneck occurs when parsing the whole page to look for the link to the sports reference website for each player in order to acquire their respective college stats. Having to parse the whole page for every player leads to a long runtime; inefficient code. A better solution would be to use another data scraping framework such as Scrapy which allows data extraction using [selectors](https://doc.scrapy.org/en/latest/topics/selectors.html). Extracting the link to sports-reference website of the page using the XPath or CSS selector would be much quicker than parsing the entire page. However as this scraper would only need to be run once it is not a major concern to optimize and rewrite another scraper.

# Utility function

In [6]:
def extract_comments(inside_html, html_id):
    comments = inside_html.findAll(text=lambda text:isinstance(text, Comment)) #data we want is commented, hence the need 
    comments = [comment.extract() for comment in comments if 'id=\"' + html_id + '\"' in comment.extract()] #get advanced stats table
    return bs(comments[0], 'html.parser')

def read_url_into_soup(url):
    next_page = urllib.request.urlopen(url).read() # goes to player page
    return bs(next_page, 'html.parser')

In [12]:
f = open("data/nba_all.csv", "w") 
f2 = open("data/college_all.csv", "w") 
writer = csv.writer(f)
writerf = csv.writer(f2)    

for year in range(1996, 2017):
    url = "https://www.basketball-reference.com/draft/NBA_"+ str(year) + ".html"
#     req = urllib.request.Request(url)
#     html = read_url_into_soup(req)
    html = read_url_into_soup(url)
#     response = urllib.request.urlopen(req)
#     html = response.read()
#     html = bs(html, 'html.parser')

    
    for row in html.table.tbody.findAll("tr"):

        if (row.find("td")) is None:
            print("END OF YEAR: "+ str(year) +"\n")  # reach end of file
            break
            
        seasons = row.find("td", {"data-stat" : "seasons"} )
        
        if (not seasons.get_text() or  #if player did not play in NBA after being drafted
        int(seasons.get_text()) < 2 or  # if less than 2 seasons played, skip
        len(row.find("td", {"data-stat" : "college_name"}) ) < 1): #if player played in euroleague
            continue
            
        player = row.find("td",{"data-stat" : "player"})
        print(player.get_text())
        inside_html = read_url_into_soup("https://www.basketball-reference.com/" + player.a['href'])
#         next_page = urllib.request.urlopen("https://www.basketball-reference.com/" + player.a['href']).read() # goes to player page
#         inside_html = bs(next_page, 'html.parser')
        
        advanced = extract_comments(inside_html, 'advanced')
        
        # STATS
          
        out = [player.string]
        cols = ['per', 'ws', 'ws_per_48', 'bpm', 'vorp'] # stats to include
        for col in cols:
            out.append(advanced.findAll('td', {'data-stat' : col})[0].string) #include both 1st and 2nd year
            out.append(advanced.findAll('td', {'data-stat' : col})[1].string) 
        writer.writerow(out)
        
        for i in inside_html.findAll("a"): #inefficient way of finding the url for college stats
            if "College Basketball" in str(i):
                coll_url = i['href']
                break
              
        coll_html = read_url_into_soup(coll_url)
#         coll_page = urllib.request.urlopen(coll_url).read()
#         coll_html = bs(coll_page, 'html.parser')

        coll_advanced = extract_comments(coll_html, 'players_advanced')
#         coll_pg = extract_comments(coll_html, 'players_per_game')
        
        #input stats
        
        ws = coll_advanced.findAll('td', {'data-stat' : 'ws'})[-2].string #obtain final year college stats ws and ws40
        ws40 = coll_advanced.findAll('td', {'data-stat' : 'ws_per_40'})[-2].string
        
        sos = coll_html.findAll('td', {'data-stat' : 'sos'})[-2].string
        writerf.writerow([player.string, ws, ws40, sos])
            
f.close()
f2.close()    

Allen Iverson
9.83
Marcus Camby
8.92
Shareef Abdur-Rahim


KeyboardInterrupt: 

## PYTHON 2.7 CODE BELOW

Do not remove in case


In [13]:
url = "https://www.basketball-reference.com/draft/NBA_2016.html"
f = urllib.urlopen(url).read()
html = bs(f, 'html.parser')
with open("2016.txt","w") as fp:
    for row in html.table.findAll("td", {"data-stat" : "player"}):
        print(row.get_text())
        next_page = urllib.urlopen("https://www.basketball-reference.com/" + row.a['href']).read()
        inside_html = bs(next_page, 'html.parser')
        comments = inside_html.findAll(text=lambda text:isinstance(text, Comment)) #data we want is commented, hence the need 
        comments = [comment.extract() for comment in comments if 'id=\"advanced\"' in comment.extract()]
        advanced = bs(comments[0], 'html.parser')
        print(advanced.find('td', {'data-stat' : 'ws'}))
        

    

Ben Simmons
<td class="right " data-stat="ws">6.3</td>
Brandon Ingram


KeyboardInterrupt: 